In [6]:
#安装 libsndfile
'''
cd /home/ec2-user/SageMaker/shouqi/libsndfile-1.0.28
./configure --prefix=/usr    \
            --disable-static \
            --docdir=/usr/share/doc/libsndfile-1.0.28 &&
make && sudo make install
'''
# !pip install asteroid -i https://pypi.tuna.tsinghua.edu.cn/simple/
# !pip install librosa==0.7.2 -i https://pypi.tuna.tsinghua.edu.cn/simple/

import os
import subprocess
import sagemaker
import torch
from sagemaker import get_execution_role
from helper.utils import *
import argparse
import librosa
import torch
from scipy.signal import lfilter
import numpy as np
import scipy.io.wavfile
import soundfile as sf
import fnmatch, os, warnings
import json
from code.pred_utils import *

sagemaker_session = sagemaker.Session()
_role = get_execution_role()

s3_path='s3://sagemaker-cn-northwest-1-969422986683/shouqi' #上传路径
sample_rate=16000 #采样率


## 准备数据

In [3]:
def generate_data(in_dir,out_dir,out_filename,ttype): #生成array以及meta_json
    file_infos = []
    in_dir = os.path.abspath(in_dir)
    wav_list = os.listdir(in_dir)
    for wav_file in wav_list:
        if not wav_file.endswith('.wav'):
            continue
        wav_path = os.path.join(in_dir, wav_file)
        samples, _ = librosa.load(wav_path, sr=sample_rate)
        fake_wav_path = os.path.join('/opt/ml/input/data', ttype, out_filename, wav_file)
        file_infos.append((fake_wav_path, len(samples)))
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    with open(os.path.join(out_dir, out_filename + '.json'), 'w') as f:
        json.dump(file_infos, f, indent=4)
        
generate_data('train_sample_v6/tr/mix_both','train_sample_v6/tr','mix_both','train')
generate_data('train_sample_v6/tr/s1','train_sample_v6/tr','s1','train')
generate_data('train_sample_v6/tr/s2','train_sample_v6/tr','s2','train')
generate_data('train_sample_v6/cv/mix_both','train_sample_v6/cv','mix_both','test')
generate_data('train_sample_v6/cv/s1','train_sample_v6/cv','s1','test')
generate_data('train_sample_v6/cv/s2','train_sample_v6/cv','s2','test')


!aws s3 sync train_sample_v6/tr/ $s3_path/separate/train
!aws s3 sync train_sample_v6/cv/ $s3_path/separate/test

upload: train_sample_v6/tr/mix_both.json to s3://sagemaker-cn-northwest-1-969422986683/shouqi/separate/train/mix_both.json
upload: train_sample_v6/tr/s2.json to s3://sagemaker-cn-northwest-1-969422986683/shouqi/separate/train/s2.json
upload: train_sample_v6/tr/s1.json to s3://sagemaker-cn-northwest-1-969422986683/shouqi/separate/train/s1.json
upload: train_sample_v6/cv/mix_both.json to s3://sagemaker-cn-northwest-1-969422986683/shouqi/separate/test/mix_both.json
upload: train_sample_v6/cv/s1.json to s3://sagemaker-cn-northwest-1-969422986683/shouqi/separate/test/s1.json
upload: train_sample_v6/cv/s2.json to s3://sagemaker-cn-northwest-1-969422986683/shouqi/separate/test/s2.json


## 开始训练和部署流程
### 设置训练和验证数据路径

In [4]:
train_inputs="{}/separate/train/".format(s3_path)
test_inputs="{}/separate/test/".format(s3_path)

### 利用sagemaker pytorch进行训练

In [5]:
from sagemaker.pytorch import PyTorch

dprnn_estimator = PyTorch(entry_point='dprnn-train-deploy.py',
                            source_dir='code',
                            train_instance_type='ml.p3.2xlarge',
                            train_instance_count=1,
                            role=_role,
                            train_use_spot_instances=True, #save up to 90% training cost
                            train_max_wait=86400,
                            framework_version='1.5.1',
                            hyperparameters = {'epochs': 20})
dprnn_estimator.fit({'train': train_inputs,
                       'test': test_inputs})

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2020-08-16 13:40:24 Starting - Starting the training job...
2020-08-16 13:40:27 Starting - Launching requested ML instances......
2020-08-16 13:41:35 Starting - Preparing the instances for training..

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-56c022ba9ec2>", line 13, in <module>
    'test': test_inputs})
  File "/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/sagemaker/estimator.py", line 497, in fit
    self.latest_training_job.wait(logs=logs)
  File "/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/sagemaker/estimator.py", line 1109, in wait
    self.sagemaker_session.logs_for_job(self.job_name, wait=True, log_type=logs)
  File "/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/sagemaker/session.py", line 3028, in logs_for_job
    time.sleep(poll)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/pytorch_p

KeyboardInterrupt: 

### 将训练好的模型进行部署

In [ ]:
# Deploy my estimator to a SageMaker Endpoint and get a Predictor
predictor = dprnn_estimator.deploy(instance_type='ml.g4dn.xlarge',
                                     initial_instance_count=1)

In [ ]:
# 或者已有训练结果,创建endpoint
from sagemaker.pytorch.model import PyTorchModel
import time 

#修改为模型路径
model_path='s3://sagemaker-cn-northwest-1-969422986683/pytorch-training-2020-06-26-06-37-38-883/output/model.tar.gz' 
model_name='shouqiDPRNN'

sm_model = PyTorchModel(model_data=model_path,
                     role=_role,
                     name=model_name,
                        source_dir='code',
                    entry_point='code/dprnn-train-deploy.py',
                     py_version="py3",
                     framework_version="1.3.0",
                     sagemaker_session=sagemaker_session)
endpoint_name = f'endpoint-{model_name}-' \
                    + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

predictor = sm_model.deploy(endpoint_name=endpoint_name,
                         instance_type='ml.g4dn.xlarge',
                         initial_instance_count=1)


In [7]:
#或者已有endpoint restore predictor
import sagemaker
from sagemaker.pytorch.model import PyTorchPredictor

#修改为endpoint_name
endpoint_name='endpoint-shouqiDPRNN-2020-08-16-07-42-14'
predictor=PyTorchPredictor(endpoint_name, sagemaker_session=sagemaker_session)

### 在部署好的模型上进行推理

In [10]:
%%time
instance_time = 10

filename = './test.wav'

def wav_clip(filename,instance_time=instance_time,sample_rate=sample_rate):
    T = instance_time*sample_rate
    fs, x = scipy.io.wavfile.read(filename)
    emp = np.zeros(T)
    if x.shape[0]<=T:
        emp[:x.shape[0]] = x
        return emp
    else:
        return x[:T]
    
x=wav_clip(filename)
x.shape

CPU times: user 0 ns, sys: 1.58 ms, total: 1.58 ms
Wall time: 2.4 ms


(160000,)

In [ ]:
%%time
class RawWavDataset(torch.utils.data.Dataset):
    
    def __init__(self, filename, fs=16000, frame_size=1600):
        input_np_arr, _ = preprocess_wav(filename, fs = fs)       
        self.data = librosa.util.frame(input_np_arr, 
                                       frame_length=frame_size, 
                                       hop_length=frame_size, 
                                       axis=0)
        self.nframes = len(self.data)
        
    def __len__(self):
        return self.nframes
    
    def __getitem__(self, idx):
        return torch.as_tensor(self.data[idx])

filename = '300_AUDIO.wav'
# filename = 'test.wav'
BATCH_SIZE = 8
num_workers = 2

def collate(batches):
    lengths = torch.tensor([len(item) for item in batches])
    # perform padding and convert to tensor
    return pad_list(batches, 0), lengths

dloader = torch.utils.data.DataLoader(RawWavDataset(filename), 
                                      batch_size=BATCH_SIZE, 
                                      num_workers=num_workers,
                                      collate_fn = collate)


def separate_process(xs, predictor, mix_lengths):
    # Forward
    response = predictor.predict(xs)#mixture_np
    response = torch.from_numpy(response).cpu() 
    # Remove padding and flat
    flat_estimate = remove_pad(response, mix_lengths)
    return flat_estimate

s1,s2 = [],[]
for batch, length in tqdm(dloader):
    est = separate_process(batch, predictor, length)
    for b in range(len(est)):
        s1.extend([est[b][0]])
        s2.extend([est[b][1]])

s1 = np.concatenate(s1)
s2 = np.concatenate(s2)
write(s1, './complete_split_s1.wav')
write(s2, './complete_split_s2.wav')
print('saved')










  0%|          | 0/811 [00:00<?, ?it/s]








  0%|          | 1/811 [00:00<04:00,  3.37it/s]








  0%|          | 2/811 [00:00<03:29,  3.85it/s]








  0%|          | 3/811 [00:00<03:06,  4.34it/s]








  0%|          | 4/811 [00:00<02:50,  4.74it/s]








  1%|          | 5/811 [00:00<02:37,  5.11it/s]








  1%|          | 6/811 [00:01<02:29,  5.39it/s]








  1%|          | 7/811 [00:01<02:23,  5.61it/s]








  1%|          | 8/811 [00:01<02:19,  5.76it/s]








  1%|          | 9/811 [00:01<02:21,  5.69it/s]








  1%|          | 10/811 [00:01<02:17,  5.81it/s]








  1%|▏         | 11/811 [00:01<02:16,  5.87it/s]








  1%|▏         | 12/811 [00:02<02:14,  5.95it/s]








  2%|▏         | 13/811 [00:02<02:12,  6.00it/s]








  2%|▏         | 14/811 [00:02<02:12,  6.03it/s]








  2%|▏         | 15/811 [00:02<02:11,  6.07it/s]








  2%|▏         | 16/811 [00:02<02:10,  6.10it/s]








  2%|▏         | 17/811 [00:02<02:10,  6

In [12]:
%%time
# !pip install librosa==0.7.2 -i https://pypi.tuna.tsinghua.edu.cn/simple/
# !pip install numba==0.50.0 -i https://pypi.tuna.tsinghua.edu.cn/simple/
# !pip install torchaudio==0.3.0 -i https://pypi.tuna.tsinghua.edu.cn/simple/
# !pip install torch==1.3.1 -i https://pypi.tuna.tsinghua.edu.cn/simple/

import librosa
from tqdm import tqdm
from helper.utils import *
from code.pred_utils import *


def preprocess_wav(filename, fs = 16000):
    x, fs = librosa.load(filename, sr =fs)
    x_mirror = x
    asl_level = -26.0
    y = x + x_mirror
#     y = y/10**(asl_meter(y, fs)/20) * 10**(asl_level/20)
    return y, fs


def pad_list(xs, pad_value):
    n_batch = len(xs)
    max_len = max(x.size(0) for x in xs)
    pad = xs[0].new(n_batch, max_len, * xs[0].size()[1:]).fill_(pad_value)
    for i in range(n_batch):
        pad[i, :xs[i].size(0)] = xs[i]
    return pad

def remove_pad(inputs, inputs_lengths):
    """
    Args:
        inputs: torch.Tensor, [B, C, T] or [B, T], B is batch size
        inputs_lengths: torch.Tensor, [B]
    Returns:
        results: a list containing B items, each item is [C, T], T varies
    """
    results = []
    dim = inputs.dim()
    if dim == 3:
        C = inputs.size(1)
    for input, length in zip(inputs, inputs_lengths):
        if dim == 3: # [B, C, T]
            results.append(input[:,:length].view(C, -1).cpu().numpy())
        elif dim == 2:  # [B, T]
            results.append(input[:length].view(-1).cpu().numpy())
    return results

def separate_process(xs, predictor):
    # perform padding and convert to tensor
    mixtures_pad = pad_list(xs, 0)
    
    mix_lengths = torch.tensor([len(item) for item in xs]) #torch.tensor([len(a) for a in xs])
    # Forward
    response = predictor.predict(mixtures_pad)#mixture_np
    response = torch.from_numpy(response).cpu() 
    # Remove padding and flat
    flat_estimate = remove_pad(response, mix_lengths)
    return flat_estimate

filename = '300_AUDIO.wav' #(10376000,)
# filename = 'test.wav' #(10376000,)

instance_time = 1

x, fs = preprocess_wav(filename)

s_x = instance_time * fs

def batchify(input_np_arr, n = 8, frame_size = s_x):
    batch = []
    framed = librosa.util.frame(input_np_arr, frame_length=frame_size, hop_length=frame_size, axis=0)
    for i, fr in enumerate(framed):
        batch.append(torch.from_numpy(fr).float())
        if (i+1) % n == 0:
            yield batch
            batch = []
    if len(batch) > 0:
        yield batch
        
    
# s1 = []
# s2 = []
# for batch in tqdm(batchify(x)):
#     est = separate_process(batch, predictor)
#     for b in range(len(est)):
#         s1.extend([est[b][0]])
#         s2.extend([est[b][1]])

# s1 = np.concatenate(s1)
# s2 = np.concatenate(s2)
# write(s1, './complete_split_s1.wav')
# write(s2, './complete_split_s2.wav')
# print('saved')

CPU times: user 44.5 ms, sys: 25.4 ms, total: 69.8 ms
Wall time: 192 ms


In [ ]:
# sagemaker_session.delete_endpoint(predictor.endpoint)
x.shape
# for start in range(0, l_x-s_x, s_x):
    
#     flat_estimate = separate_process(x[start:start+s_x], predictor)
#     s1 = np.concatenate([s1, flat_estimate[0]])
#     s2 = np.concatenate([s2, flat_estimate[1]])
#     gl_i = start+s_x
    
    
# if gl_i < l_x:
#     flat_estimate = separate_process(x[gl_i:], predictor)
#     s1 = np.concatenate([s1, flat_estimate[0]])
#     s2 = np.concatenate([s2, flat_estimate[1]])
# write(s1, './complete_split_s1.wav')
# write(s2, './complete_split_s2.wav')
# print('saved')